## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [8]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt5


nx = 9
ny = 6

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

def calibration(images):
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.
    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        img_size = (img.shape[1], img.shape[0])
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)    
        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        objp = np.zeros((6*9,3), np.float32)
        objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    return mtx, dist

cam_mtx, cam_dist = calibration(images)

def warp(img):
    # performs the camera calibration, image distortion correction and 
    # returns the undistorted image
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    gray = cv2.cvtColor(undist,cv2.COLOR_BGR2GRAY)
    offset = 100 # offset for dst points
    gray_size = (gray.shape[1], gray.shape[0])
    # For source points I'm grabbing the outer four detected corners
    src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough for our purposes
    dst = np.float32([[offset, offset], [gray_size[0]-offset, offset], 
                      [gray_size[0]-offset, gray_size[1]-offset], 
                      [offset, gray_size[1]-offset]])
    # Given src and dst points, calculate the perspective transf
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(undist, M, gray_size)
    return M, warped
        #cv2.imshow('img',img)
        #cv2.waitKey(500)

#cv2.destroyAllWindows()


In [3]:
def mask_image(image):
    sw_x = image.shape[1] * .01
    sw_y = image.shape[0]
    nw_x = int(image.shape[1] * .35)
    nw_y = int(image.shape[0] * .65)
    ne_x = int(image.shape[1] * .65)
    ne_y = int(image.shape[0] * .65)
    se_x = int(image.shape[1] + (image.shape[1] * .1))
    se_y = int(image.shape[0])


    vertices = np.array([[(sw_x, sw_y),
                          (nw_x,nw_y),
                          (ne_x,ne_y),
                          (se_x,se_y)]], dtype=np.int32)



    mask = np.zeros_like(image)
    # defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    # filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    # returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def source():
    src = np.float32([
        [475,530],
        [830,530],
        [130,720],
        [1120,720]
    ])
    return src

def destination():
    src = np.float32([
        [365,540],
        [990,540],
        [320,720],
        [960,720]
    ])
    return src

## And so on and so forth...

In [30]:
def pipeline(img, s_thresh=(75, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hsv[:,:,2]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    #color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    return combined_binary


In [10]:
image = cv2.imread('../test_images/test1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
masked = mask_image(image)
undistorted_dash = cv2.undistort(masked, cam_mtx, cam_dist, None, cam_mtx)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
f.tight_layout()
ax1.imshow(image,cmap='gray')
ax1.set_title('Original Image', fontsize=12)
ax2.imshow(undistorted_dash)
ax2.set_title('Undistorted Image', fontsize=12)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [13]:
image = cv2.imread('../test_images/test2.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
masked = mask_image(image)
undistorted_dash = cv2.undistort(masked, cam_mtx, cam_dist, None, cam_mtx)
m = cv2.getPerspectiveTransform(source(), destination())
m_inverse = cv2.getPerspectiveTransform(destination(), source())
image_size = (undistorted_dash.shape[1], undistorted_dash.shape[0])
warped = cv2.warpPerspective(undistorted_dash, m, image_size, flags=cv2.INTER_LINEAR)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
f.tight_layout()
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=12)
ax2.imshow(warped)
ax2.set_title('warped Image', fontsize=12)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [31]:
mage = cv2.imread('../test_images/test3.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
masked = mask_image(image)
undistorted_dash = cv2.undistort(masked, cam_mtx, cam_dist, None, cam_mtx)
m = cv2.getPerspectiveTransform(source(), destination())
m_inverse = cv2.getPerspectiveTransform(destination(), source())
image_size = (undistorted_dash.shape[1], undistorted_dash.shape[0])
warped = cv2.warpPerspective(undistorted_dash, m, image_size, flags=cv2.INTER_LINEAR)
pipe = pipeline(warped)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
f.tight_layout()
ax1.imshow(warped)
ax1.set_title('Undistorted Image', fontsize=12)
ax2.imshow(pipe,cmap='gray')
ax2.set_title('Threshold Image', fontsize=12)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [28]:
image = cv2.imread('../test_images/test3.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
masked = mask_image(image)
undistorted_dash = cv2.undistort(masked, cam_mtx, cam_dist, None, cam_mtx)
m = cv2.getPerspectiveTransform(source(), destination())
m_inverse = cv2.getPerspectiveTransform(destination(), source())
image_size = (undistorted_dash.shape[1], undistorted_dash.shape[0])
warped = cv2.warpPerspective(undistorted_dash, m, image_size, flags=cv2.INTER_LINEAR)
pipe = pipeline(warped)
histogram = np.sum(pipe[pipe.shape[0] // 2:, :], axis=0)
out_img = np.dstack((pipe, pipe, pipe)) * 255
#
plt.plot(histogram)
plt.show(block=True)

In [26]:
# window settings
window_width = 50 
window_height = np.int(pipe.shape[0]/9) # Break image into vertical layers since image height is 720
margin = 100 # How much to slide left and right for searching

def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(image, window_width, window_height, margin):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(pipe[int(3*pipe.shape[0]/4):,:int(pipe.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(pipe[int(3*pipe.shape[0]/4):,int(pipe.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(pipe.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(pipe.shape[0]/window_height)):
	    # convolve the window into the vertical slice of the image
	    image_layer = np.sum(pipe[int(pipe.shape[0]-(level+1)*window_height):int(pipe.shape[0]-level*window_height),:], axis=0)
	    conv_signal = np.convolve(window, image_layer)
	    # Find the best left centroid by using past left center as a reference
	    # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
	    offset = window_width/2
	    l_min_index = int(max(l_center+offset-margin,0))
	    l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
	    l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
	    # Find the best right centroid by using past right center as a reference
	    r_min_index = int(max(r_center+offset-margin,0))
	    r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
	    r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
	    # Add what we found for that layer
	    window_centroids.append((l_center,r_center))

    return window_centroids

window_centroids = find_window_centroids(pipe, window_width, window_height, margin)

# If we found any window centers
if len(window_centroids) > 0:

    # Points used to draw all the left and right windows
    l_points = np.zeros_like(pipe)
    r_points = np.zeros_like(pipe)

    # Go through each level and draw the windows 
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas
        l_mask = window_mask(window_width,window_height,pipe,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,pipe,window_centroids[level][1],level)
        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
        r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage = np.array(cv2.merge((pipe,pipe,pipe)),np.uint8) # making the original road pixels 3 color channels
    output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
 
# If no window centers found, just display orginal road image
else:
    output = np.array(cv2.merge((pipe,pipe,pipe)),np.uint8)

# Display the final results
plt.imshow(output)
plt.title('window fitting results')
plt.show()